## Extract Raw HTML

In [1]:
from bs4 import BeautifulSoup

# Load the HTML file
with open('/home/markremmey/ixbrl/data/output_file.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Extract all <div> elements with class "page"
divs = soup.find_all('div', class_='page')

# Convert the extracted divs to a string
divs_html = ''.join(str(div) for div in divs)

# Print or use the extracted HTML as needed
print(divs_html)


<div class="page" id="v1-page1-body">
<p class="heading" id="v1-page1-p_0" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; margin: 0in;">Company registration number: <ix:nonnumeric contextref="c_0" id="v1-page1-p_0-tag44249433" name="ie-common:CompaniesRegistrationOfficeNumber">234567</ix:nonnumeric></p>
<p id="v1-page1-p_1" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; line-height: 12pt; margin: 0in;">
<br/>
</p>
<p class="heading" id="v1-page1-p_2" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; margin: 0in;">
<ix:nonnumeric contextref="c_0" id="v1-page1-p_2-tag44249135" name="bus:EntityCurrentLegalOrRegisteredName">XYZ Limited</ix:nonnumeric>
</p>
<p class="heading" id="v1-page1-p_3" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; margin: 0in;"><ix:nonnumeric contextref="c_0" id="v1-page1-p_3-tag44249136" name="bus:ReportTitle">Annual Report for the financial

In [2]:
soup = BeautifulSoup(divs_html, "html.parser")
    
pages_data = []
pages = soup.find_all("div", class_="page")

In [3]:
pages[0]

<div class="page" id="v1-page1-body">
<p class="heading" id="v1-page1-p_0" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; margin: 0in;">Company registration number: <ix:nonnumeric contextref="c_0" id="v1-page1-p_0-tag44249433" name="ie-common:CompaniesRegistrationOfficeNumber">234567</ix:nonnumeric></p>
<p id="v1-page1-p_1" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; line-height: 12pt; margin: 0in;">
<br/>
</p>
<p class="heading" id="v1-page1-p_2" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; margin: 0in;">
<ix:nonnumeric contextref="c_0" id="v1-page1-p_2-tag44249135" name="bus:EntityCurrentLegalOrRegisteredName">XYZ Limited</ix:nonnumeric>
</p>
<p class="heading" id="v1-page1-p_3" style="font-family: 'Calibri\000020Light'; font-size: 12pt; font-weight: bold; margin: 0in;"><ix:nonnumeric contextref="c_0" id="v1-page1-p_3-tag44249136" name="bus:ReportTitle">Annual Report for the financial

In [4]:
len(pages)

12

In [5]:
import json
from bs4 import BeautifulSoup

def parse_ixbrl(html):
    soup = BeautifulSoup(html, "html.parser")
    
    pages_data = []
    pages = soup.find_all("div", class_="page")
    
    for idx, page in enumerate(pages):
        # Try to use the page id as a numeric value; if unavailable, use index + 1.
        page_id = page.get("id")
        try:
            page_no = int(page_id) if page_id and page_id.isdigit() else idx + 1
        except:
            page_no = idx + 1
        
        # Extract raw text using a separator.
        text = page.get_text(separator=" ", strip=True)
        
        # Extract applied iXBRL tags within this page.
        tags_dict = {}
        for tag in page.find_all():
            if tag.name.startswith("ix:"):
                tag_key = tag.get("name")
                tag_value = tag.get_text(strip=True)
                if tag_key:
                    tags_dict[tag_key] = tag_value
        
        pages_data.append({
            "page_no": page_no,
            "raw_text": text,
            "tags": tags_dict
        })
    
    # Return the structured data as a JSON string.
    return json.dumps(pages_data, indent=2)

# --- Example usage ---
if __name__ == "__main__":
    # For demonstration purposes, here's a placeholder HTML snippet.
    
    # Parse the HTML and get the JSON formatted result.
    result_json = parse_ixbrl(divs_html)
    print("Extracted JSON data:")
    print(result_json)

Extracted JSON data:
[
  {
    "page_no": 1,
    "raw_text": "Company registration number: 234567 XYZ Limited Annual Report for the financial year ended 31 December 2023",
    "tags": {
      "ie-common:CompaniesRegistrationOfficeNumber": "234567",
      "bus:EntityCurrentLegalOrRegisteredName": "XYZ Limited",
      "bus:ReportTitle": "Annual Report for the financial year ended",
      "bus:EndDateForPeriodCoveredByReport": "31 December 2023"
    }
  },
  {
    "page_no": 2,
    "raw_text": "CONTENTS Page DIRECTORS AND OTHER INFORMATION 2 DIRECTORS' REPORT 3-5 INDEPENDENT AUDITOR'S REPORT 6-8 STATEMENT OF COMPREHENSIVE INCOME 9 STATEMENT OF FINANCIAL POSTION 10 STATEMENT OF CHANGES IN EQUITY 11 STATEMENT OF CASH FLOWS 12 NOTES TO THE FINANCIAL STATEMENTS 13-22",
    "tags": {}
  },
  {
    "page_no": 3,
    "raw_text": "DIRECTORS AND OTHER INFORMATION Board of Directors Director 1 (resigned 1 February 2023 ) Director 2 (appointed 1 February 2023 and resigned 31 March 2024 ) Director 3 

In [6]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
OPENAI_MODEL =os.getenv("AZURE_OPENAI_MODEL")
OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
OPENAI_API_EMBEDDING_MODEL = os.getenv("AZURE_OPENAI_API_EMBEDDING_MODEL")

print(f"OPENAI_API_VERSION: ", OPENAI_API_VERSION)
print(f"OPENAI_API_KEY: ", OPENAI_API_KEY)
print(f"OPENAI_API_BASE: ", OPENAI_API_BASE)


OPENAI_API_VERSION:  2024-05-01-preview
OPENAI_API_KEY:  3vdkwPbeKhjmEvsr337Awz2p29gLcG5V06WKyvAzxaTIwDhEuGJkJQQJ99BBACYeBjFXJ3w3AAABACOG2Gjy
OPENAI_API_BASE:  https://oai0-rzw5g6zn6nuug.openai.azure.com/


In [7]:
from utils.azure_openai import run_prompt

OPENAI_API_KEY:  3vdkwPbeKhjmEvsr337Awz2p29gLcG5V06WKyvAzxaTIwDhEuGJkJQQJ99BBACYeBjFXJ3w3AAABACOG2Gjy


In [15]:
system_prompt = """Please review the raw text and applied tags and see if you can identify any potential issues. Perhaps the dates don't make sense, or something seems incorrect about the address, or something else. THe following was tagged by audit reviewers in the iXBRL format.

Do not consider your known training data or general knowledge, only look at the information presented to you to make your determination. It is ok if date's are beyond your training set, only mark an issue if they are inconsistent with the information provided in the text.
"""

user_prompt = """Here is the JSON showing the pages and applied tags. i Have only provided two entries, but I want you to review all of the entries. Please respond in the following format:

##### START OF EXAMPLE #######

Raw HTML: `{
    "page_no": 11,
    "raw_text": "6 Loss on ordinary activities before taxation 2023 2022 \u20ac \u20ac Loss before taxation is stated after charging the following: Auditor's remuneration 6,500 8,550 None of the Directors received any remuneration in respect of services to the Company during the year or in the previous year. The Company had no employees during the year or in the previous year. The auditor's remuneration for the previous financial period was borne by an associated group company. 7 Tax charge on profit on ordinary activities 2023 \u20ac 2022 \u20ac (a) Analysis of tax charge in the year Irish corporation tax charge on profit for the year - - Adjustments in respect of prior years - - Current charge for the year - - (b) Factors affecting tax charge for the year The current tax charge differs from the charge that would result from applying the standard rate of Irish corporation tax to the profit/(loss) on ordinary activities before tax. The differences are explained below: 2023 2022 \u20ac \u20ac Gain/(Loss) on ordinary activities before tax 32,626,629 ( 4,089,447 ) Profit/(Loss) on ordinary activities multiplied by 12.5 % 4,078,329 ( 511,181 ) Effects of: Exempt items ( 5,580,156 ) - Trading losses carried forward 1,501,827 - Total tax charge for the year - - Losses: Trading loss for the period (12,014,617) (4,089,447) Trading loss carried forward (16,104,064) (4,089,447) A deferred tax asset of \u20ac2,013,008 (2022: \u20ac511,181) relating mostly to taxable losses carried forward has not been recognised due to uncertainty surrounding the future profitability of the Company. 8 Investment in subsidiaries 2023 2022 \u20ac \u20ac At beginning of year 124,122,608 3,626,863 Additions - 120,495,745 Disposals ( 2,484,619 ) - At end of year 121,637,989 124,122,608 On 31 st December 2023, the Group signed a sale and purchase agreement to sell its subsidiary in Subsidiary 6, a public limited liability company incorporated under the laws of the United States of America for $52,100,000 (\u20ac47,125,865) resulting in a gain on sale of subsidiary of \u20ac44,641,246. The Company acquired Subsidiary 5 on 4 May 2022 for \u20ac 120,495,745 . In the opinion of the Directors the carrying value of the investment in subsidiary companies does not differ significantly from the recoverable value. The following are the details of the subsidiaries of the Company in accordance with Section 314 of the Companies Act 2014. Company Registered office % Owned Principal activity Subsidiary 1 (formerly Subsidiary 1) Address 1, Address 3, Address 2, County 100 % Provision of corporate financial and managed services Subsidiary 2 Address 1, Address 2, Address 3 Country 1, Pin Code 100 % Fiscal and tax advice and fiscal representation of individuals and companies, management of properties and representation of the interests of the property investors and administration of companies. Subsidiary 3 Address 1, Address 3, Address 2, County 100 % The provision of regulated fund services in Ireland Subsidiary 4 (formerly Subsidiary 4) Address 1, Address 3, Address 2, County 100 % Provision of corporate trustee services Subsidiary 5 Address 1, Address 2, City, Country 100 % The provision of investment management services, this includes acting as an Alternative Investment Fund Manager (\"AIFM\") and UCITS Management Company. 9 Debtors - amounts falling due within one year 2023 2022 \u20ac \u20ac Amounts due from group companies 461,011 - Debtors - amounts falling due within one year 461,011 - Amounts due from group companies are unsecured, have no fixed date of repayment and are repayable on demand. Interest is charged on group balances with a single counterparty in excess of an average balance of \u20ac1m in the year, at a rate of 5% except if otherwise agreed with the counterparty. Interest receivable on group balances in 2023 totalled \u20acnil (2022: \u20acnil). 10 Creditors - amounts falling due within one year 2023 2022 \u20ac \u20ac Amounts due to group companies 90,415,326 125,089,631 Accrued Expenses 26,166 2,917 Bank overdraft 706 - Creditors - amounts falling due within one year 90,442,198 125,092,548 Amounts due to group companies are unsecured, have no fixed date of repayment and are repayable on demand. Interest is charged on group balances with a single counterparty in excess of an average balance of \u20ac1m in the year, at a rate of 5% except if otherwise agreed with the counterparty. Interest payable on group balances in 2023 totalled \u20ac11,980,834 (2022: \u20ac4,090,010). 11 Reconciliation of movement in equity shareholders' funds 2023 2022 \u20ac \u20ac Equity shareholders' funds at beginning of year (3,454,345) 635,102 Gain/(Loss) for the financial year 32,626,629 ( 4,089,447 ) Equity shareholders' funds at end of year 29,172,284 (3,454,345) 12 Reserves Retained (losses)/earnings 2023 2022 \u20ac \u20ac At beginning of year ( 4,089,445 ) 2 Profit/(Loss) for the financial year 32,626,629 ( 4,089,447 ) At end of year 28,537,184 ( 4,089,445 ) Share Capital Called up share capital 2023 2022 \u20ac \u20ac Authorised 100,000 ordinary shares of \u20ac 1 each 100,000 100,000 Allotted, called up and fully paid 101 ordinary shares of \u20ac 1 each 101 101 Share Premium There was no movement in share premium during the year ended 31 December 2023 (2022 & 2023: \u20ac 2,484,517 ). Capital Contribution Reserve Within reserves, there is \u20ac 635,000 (2022: \u20ac 635,000 ) capital contribution reserve. The capital contribution was made to the Company on 29 November 2018 by Immediate parent (formerly Immediate parent) in order to assist with the Company to meet regulatory requirements in respect of its capital. 13 Controlling parties The Company's immediate parent is Immediate parent . The ultimate controlling party of the XYZ Group is Ultimate parent . Within the XYZ Group consolidated financial statements are prepared for Entity 5 the results of which include the Company. Entity 5 has its registered office at Address 1, Address 2, Address 3. Entity 5 is controlled by Ultimate parent. 14 Related party disclosures Transactions with other wholly owned subsidiaries within the group are not disclosed as the Company has taken advantage of the exemption in section 33.1A of FRS 102. 15 Events after the reporting date There were no significant events after the reporting date that require disclosure or adjustment. 16 Approval of the financial statements The board of directors approved these financial statements for issue on . We hereby certify that all documents which are required under Part 6 of the Companies Act 2014 to be annexed to the annual return, have been so annexed, and that they are true copies of the originals laid or to be laid before the relevant general meeting, or presented to the members. Signature: Signature: Director 4 Director 7 Director Director Date: 18 November 2024 Date: 18 November 2024",
    "tags": {
      "aurep:AuditFeesExpenses": "8,550",
      "direp:DirectorsRemunerationFree-textComment": "None of the Directors received any remuneration in respect of services to the Company during the year or in the previous year.",
      "core:EmployeeInformationFree-textComment": "The Company had no employees during the year or in the previous year.",
      "aurep:FeesToAuditorsFree-textComment": "The auditor's remuneration for the previous financial period was borne by an associated group company.",
      "core:CurrentTaxForPeriod": "-",
      "core:IncreaseDecreaseInCurrentTaxFromAdjustmentForPriorPeriods": "-",
      "core:TotalCurrentTaxExpenseCredit": "-",
      "core:IncomeTaxFree-textComment": "The current tax charge differs from the charge that would result from applying the standard rate of Irish corporation tax to the profit/(loss) on ordinary activities before tax.",
      "core:ProfitLossOnOrdinaryActivitiesBeforeTax": "4,089,447",
      "core:ApplicableTaxRate": "12.5",
      "core:TaxExpenseCreditApplicableTaxRate": "511,181",
      "core:TaxDecreaseIncreaseFromEffectRevenueExemptFromTaxation": "-",
      "core:TaxIncreaseDecreaseFromEffectUnrelievedTaxLossesCarriedForward": "-",
      "core:TaxTaxCreditOnProfitOrLossOnOrdinaryActivities": "-",
      "core:DeferredTaxationFree-textComment": "A deferred tax asset of \u20ac2,013,008 (2022: \u20ac511,181) relating mostly to taxable losses carried forward has not been recognised due to uncertainty surrounding the future profitability of the Company.",
      "core:InvestmentsInSubsidiaries": "120,495,745",
      "core:InvestmentsInSubsidiariesFree-textComment": "On 31stDecember 2023, the Group signed a sale and purchase agreement to sell its subsidiary in Subsidiary 6, a public limited liability company incorporated under the laws of the United States of America for $52,100,000 (\u20ac47,125,865) resulting in a gain on sale of subsidiary of \u20ac44,641,246.The Company acquired Subsidiary 5 on 4 May 2022 for \u20ac120,495,745.In the opinion of the Directors the carrying value of the investment in subsidiary companies does not differ significantly from the recoverable value.",
      "core:NameSubsidiary": "Subsidiary 5",
      "core:OwnershipInterestInSubsidiaryPercent": "100",
      "core:DescriptionNatureBusinessSubsidiary": "The provision of investment management services, this includes acting as an Alternative Investment Fund Manager (\"AIFM\") and UCITS Management Company.",
      "core:AmountsOwedByGroupUndertakings": "-",
      "core:Debtors": "-",
      "core:DebtorsFree-textComment": "Amounts due from group companies are unsecured, have no fixed date of repayment and are repayable on demand. Interest is charged on group balances with a single counterparty in excess of an average balance of \u20ac1m in the year, at a rate of 5% except if otherwise agreed with the counterparty. Interest receivable on group balances in 2023 totalled \u20acnil (2022: \u20acnil).",
      "core:AmountsOwedToGroupUndertakings": "125,089,631",
      "core:AccruedLiabilities": "2,917",
      "core:BankOverdrafts": "-",
      "core:Creditors": "125,092,548",
      "core:CreditorsFree-textComment": "Amounts due to group companies are unsecured, have no fixed date of repayment and are repayable on demand. Interest is charged on group balances with a single counterparty in excess of an average balance of \u20ac1m in the year, at a rate of 5% except if otherwise agreed with the counterparty. Interest payable on group balances in 2023 totalled \u20ac11,980,834 (2022: \u20ac4,090,010).",
      "core:ProfitLoss": "4,089,447",
      "core:Equity": "635,000",
      "bus:DescriptionShareType": "ordinary shares",
      "core:ParValueShare": "1",
      "core:NumberSharesIssuedFullyPaid": "101",
      "core:DescriptionNaturePurposeReservesWithinEquity": "Within reserves, there is \u20ac635,000(2022: \u20ac635,000) capital contribution reserve. The capital contribution was made to the Company on 29 November 2018 by Immediate parent (formerly Immediate parent) in order to assist with the Company to meet regulatory requirements in respect of its capital.",
      "core:DescriptionRelationshipsBetweenEntityParentsIncludingAnyChanges": "The Company's immediate parent isImmediate parent. The ultimate controlling party of the XYZ Group isUltimate parent. Within the XYZ Group consolidated financial statements are prepared for Entity 5 the results of which include the Company. Entity 5 has its registered office at Address 1, Address 2, Address 3. Entity 5 is controlled by Ultimate parent.",
      "core:NameParentEntity": "Immediate parent",
      "core:NameUltimateControllingPartyIfNotUltimateParent": "Ultimate parent",
      "core:DescriptionBodyAuthorisingFinancialStatements": "board",
      "bus:NameEntityOfficer": "Director 7"
    }`

  Expected Output:  `
  [
  {
    "concept_label": "Audit Fees Expenses",
    "tag_value": "8,550",
    "page_no": 11,
    "Rationale": "The auditor's remuneration is recorded as 6,500 for 2023 and 8,550 for 2022. The tag correctly reflects the 2022 amount, which matches the text.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Directors Remuneration Free-text Comment",
    "tag_value": "None of the Directors received any remuneration in respect of services to the Company during the year or in the previous year.",
    "page_no": 11,
    "Rationale": "The tagged value matches the raw text, confirming that no directors received remuneration for services to the company.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Employee Information Free-text Comment",
    "tag_value": "The Company had no employees during the year or in the previous year.",
    "page_no": 11,
    "Rationale": "The tag accurately reflects the raw text and no inconsistencies were identified.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Applicable Tax Rate",
    "tag_value": "12.5",
    "page_no": 11,
    "Rationale": "The tax rate is stated as 12.5% in the calculation of tax expense. No inconsistencies were detected.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Tax Expense Credit Applicable Tax Rate",
    "tag_value": "511,181",
    "page_no": 11,
    "Rationale": "The tax expense credit at the applicable tax rate is listed as (511,181) in the breakdown of tax calculations. This matches the tag value.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Profit (Loss) on Ordinary Activities Before Tax",
    "tag_value": "4,089,447",
    "page_no": 11,
    "Rationale": "The profit (loss) before tax is stated as (4,089,447) in 2022 but 32,626,629 in 2023. The tagged value of 4,089,447 appears to reference 2022 rather than 2023, which could be an error if the intent was to tag the latest value.",
    "Review Result": "Flagged for Review - Possible Year Mismatch"
  },
  {
    "concept_label": "Investments in Subsidiaries",
    "tag_value": "120,495,745",
    "page_no": 11,
    "Rationale": "The company acquired Subsidiary 5 on 4 May 2022 for €120,495,745, which matches the tagged value. No inconsistencies found.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Deferred Taxation Free-text Comment",
    "tag_value": "A deferred tax asset of €2,013,008 (2022: €511,181) relating mostly to taxable losses carried forward has not been recognised due to uncertainty surrounding the future profitability of the Company.",
    "page_no": 11,
    "Rationale": "The tag accurately reflects the text, and no inconsistencies were found.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Creditors - Amounts Owed to Group Undertakings",
    "tag_value": "125,089,631",
    "page_no": 11,
    "Rationale": "The text states that amounts owed to group companies are 90,415,326 for 2023 and 125,089,631 for 2022. Since the tagged value corresponds to 2022 rather than the current year, this might be an issue if the latest value was intended.",
    "Review Result": "Flagged for Review - Possible Year Mismatch"
  },
  {
    "concept_label": "Equity",
    "tag_value": "635,000",
    "page_no": 11,
    "Rationale": "The tag corresponds to the capital contribution reserve, which remained at €635,000 for both 2022 and 2023. No inconsistencies found.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Name Entity Officer",
    "tag_value": "Director 7",
    "page_no": 11,
    "Rationale": "The document states that financial statements were approved on 18 November 2024, which is a future date relative to the financial reporting period ending 31 December 2023. This suggests that the approval date may be incorrect.",
    "Review Result": "Flagged for Review - Future Date Discrepancy"
  }
]`
##### END OF EXAMPLE #######

Only print the JSON output. Do not include tags such as ```json ...```. Do not provide any additional commentary outside of the JSON format specified

Raw data:\n\n
"""

In [20]:
new_json = json.loads(response)
print(new_json)

[{'concept_label': 'Detailed Profit And Loss Turnover Revenue', 'tag_value': '-', 'page_no': 12, 'Rationale': 'The tag indicates no turnover revenue, consistent with the raw text for 2022. No discrepancies identified.', 'Review Result': 'No Issues Detected'}, {'concept_label': 'Detailed Profit And Loss Government Grant Income', 'tag_value': '-', 'page_no': 12, 'Rationale': 'The tag reflects no government grant income, which matches the raw text for both years.', 'Review Result': 'No Issues Detected'}, {'concept_label': 'Detailed Profit And Loss Other Operating Income', 'tag_value': '8,480.00', 'page_no': 12, 'Rationale': "The tag value corresponds to the 2022 'Other income' value of 8,480.00. This matches the raw text.", 'Review Result': 'No Issues Detected'}, {'concept_label': 'Detailed Profit And Loss Gross Profit (Loss)', 'tag_value': '-', 'page_no': 12, 'Rationale': 'The tag correctly represents an unspecified gross profit or loss, consistent with the blank values in the raw text.'

In [21]:
import json
result_list = []

for page_json in json.loads(result_json):
    response = run_prompt(system_prompt, user_prompt + json.dumps(page_json))
    result_list.extend(json.loads(response))
    print("Response: ", response)

# Convert JSON list to CSV and save to file
import pandas as pd
df = pd.DataFrame(result_list)
df.to_csv('ixbrl_review_results.csv', index=False)


Response:  [
  {
    "concept_label": "Companies Registration Office Number",
    "tag_value": "234567",
    "page_no": 1,
    "Rationale": "The tag value matches the company registration number stated in the raw text.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Entity Current Legal Or Registered Name",
    "tag_value": "XYZ Limited",
    "page_no": 1,
    "Rationale": "The tag value matches the legal or registered name of the company as stated in the raw text.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "Report Title",
    "tag_value": "Annual Report for the financial year ended",
    "page_no": 1,
    "Rationale": "The tag value accurately reflects the raw text's reference to the report title.",
    "Review Result": "No Issues Detected"
  },
  {
    "concept_label": "End Date For Period Covered By Report",
    "tag_value": "31 December 2023",
    "page_no": 1,
    "Rationale": "The tag value corresponds to the financial yea

## Match Concept Labels to Original CSVs

In [18]:
import pandas as pd

# Specify the file path and sheet name or index
file_path = './data/XYZ.xlsx'
sheet_name = 'Filing Details'  # You can also use the sheet index, e.g., sheet_name=0 for the first sheet

# Read the specified sheet into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Display the first few rows of the DataFrame
print(df.head())


   Order          ID Tag Value Changed Confirmed Line Item Description  \
0    1.0  44249433.0                No       Yes                   NaN   
1    2.0  44249135.0                No       Yes                   NaN   
2    3.0  44249136.0                No       Yes                   NaN   
3    4.0  44249137.0                No       Yes                   NaN   
4    5.0  44249138.0                No       Yes                   NaN   

                             Concept Label  \
0     Companies Registration Office number   
1  Entity current legal or registered name   
2                             Report title   
3    End date for period covered by report   
4                   Name of entity officer   

                                        Concept Name Concept Balance  \
0  {https://xbrl.frc.org.uk/ireland/common/2023-0...             NaN   
1  {http://xbrl.frc.org.uk/cd/2023-01-01/business...             NaN   
2  {http://xbrl.frc.org.uk/cd/2023-01-01/business...          